In [7]:
# weird bug where you need the latest bleeding edge version of transformers

%pip uninstall transformers
%pip install git+https://github.com/huggingface/transformers.git

Found existing installation: transformers 4.44.0.dev0
Uninstalling transformers-4.44.0.dev0:
  Would remove:
    /Users/sueszli/.asdf/installs/python/3.11.9/bin/transformers-cli
    /Users/sueszli/.asdf/installs/python/3.11.9/lib/python3.11/site-packages/transformers-4.44.0.dev0.dist-info/*
    /Users/sueszli/.asdf/installs/python/3.11.9/lib/python3.11/site-packages/transformers/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection 

model_id = "IDEA-Research/grounding-dino-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)
# Check for cats and remote controls
# VERY important: text queries need to be lowercased + end with a dot
text = "a cat. a remote control."

inputs = processor(images=image, text=text, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

results = processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    box_threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)


/Users/sueszli/.asdf/installs/python/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ValueError: The checkpoint you are trying to load has model type `grounding-dino` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.